In [53]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import requests
import mwparserfromhell
import urllib.request
import json
import re

url = "https://wowpedia.fandom.com/wiki/Major_characters"
html = urlopen(url).read()
soup = BeautifulSoup(html, features="html.parser")



In [42]:
characters = []

games = ['WC1','WC2','WC2X','WC3','WC3X','WoW','WoWBC','WoWWotLK','WoWC','WoWMoP','WoWWoD','WoWL','WoWBfA','WoWS','WoWD']

for s in soup.find_all('p'):
    text = s.text
    if "—" in text:
        
        d = {}
        
        t = text.replace("\n", "").split("—")

        name = t[0].strip().replace(" ", "_") #.replace(" ", "")

        expansions = t[1].replace(" ", "").split(",")
        expansions = [1 if i in expansions else 0 for i in games]
        
        if 1 in expansions:
            d['name'] = name
            for i,g in enumerate(games):
                d[g] = expansions[i]
                
            characters.append(d)


In [43]:
df = pd.DataFrame(characters)

In [44]:
names = list(df['name'])
itemus = names[0]

url = f"https://wowpedia.fandom.com/api.php?action=query&prop=revisions&titles={itemus}&rvslots=*&rvprop=content&formatversion=2&format=json"
response = requests.get(url)
 
itemdetail = response.json()
itemdetail['query']['pages'][0]['revisions'][0]['slots']['main']['content']



'{{Foralt}}\n{{npcbox\n| name = Terokk\n| image = Terokk.jpg\n| titles = The Talon King\n| gender = Male\n| level = 72\n| type = Elite\n| race = Arakkoa\n| creature = Undead\n| faction = Combat\n| aggro = {{aggro|-1|-1}}\n| foccupation = Talon King of the Arakkoa, King of the Outcasts and Founder of Skettis\n| status = Deceased (lore)<br/> Killable {{bc-inline}}\n| location = [[Terokk\'s Rest]] in [[Skettis]], [[Terokkar Forest]]\n| relatives = [[Lithic]] (daughter)\n}}\n\n\'\'\'Terokk\'\'\' was the greatest king of the [[arakkoa]]. Terokk was associated with the goddess [[Rukhmar]] and believed to be her incarnation, though this was not actually the case. Terokk was the arakkoa\'s greatest hero who was betrayed by his own allies and afflicted with the Curse of Sethe. Even in this state, however, he sought to aid his fellow refugees by building the city of [[Skettis]] with his own claws. However the curse soon began to drive him to madness, and he was sealed away by his Talonpriests in

In [45]:
t = names[0]

response = requests.get(
     'https://wowpedia.fandom.com/api.php',
         params={
         'action': 'query',
         'format': 'json',
         'titles': '{}'.format(t),
         'prop': 'revisions',
         'rvprop': 'content',
     }
 ).json()


page = next(iter(response['query']['pages'].values()))

wikicode = page['revisions'][0]['*']
parsed_wikicode = mwparserfromhell.parse(wikicode).strip_code()


In [46]:
parsed_wikicode

'Terokk was the greatest king of the arakkoa. Terokk was associated with the goddess Rukhmar and believed to be her incarnation, though this was not actually the case. Terokk was the arakkoa\'s greatest hero who was betrayed by his own allies and afflicted with the Curse of Sethe. Even in this state, however, he sought to aid his fellow refugees by building the city of Skettis with his own claws. However the curse soon began to drive him to madness, and he was sealed away by his Talonpriests in the hopes of rejuvenating him. He now waits in the shadows to be called back into the physical world and face the enemies of his people. \n\nThe heretical Talon King Ikiss claims he is Terokk returned. \n\nTerokk also has an avatar in the form of a gigantic kaliri, unleashed on Shattrath when all of his plans are thwarted by adventurers at the behest of the Skettis refugees of the Lower City. \n\nBiography\nOn Draenor\nthumb|left|Terokk doing battle with the Bloodmane saberon, before becoming cu

In [50]:
baseurl = "https://wowpedia.fandom.com/api.php?"
action = "action=query"
title = "titles={}"
content = "prop=revisions&rvprop=content"
dataformat ="format=json"

query = "{}{}&{}&{}&{}".format(baseurl, action, content, title.format(t), dataformat)
wikiresponse = urllib.request.urlopen(query)
wikidata = wikiresponse.read()
wikitext = wikidata.decode('utf-8')

In [52]:
wikitext

'{"batchcomplete":"","warnings":{"main":{"*":"Subscribe to the mediawiki-api-announce mailing list at <https://lists.wikimedia.org/postorius/lists/mediawiki-api-announce.lists.wikimedia.org/> for notice of API deprecations and breaking changes."},"revisions":{"*":"Because \\"rvslots\\" was not specified, a legacy format has been used for the output. This format is deprecated, and in the future the new format will always be used."}},"query":{"pages":{"86839":{"pageid":86839,"ns":0,"title":"Terokk","revisions":[{"contentformat":"text/x-wiki","contentmodel":"wikitext","*":"{{Foralt}}\\n{{npcbox\\n| name = Terokk\\n| image = Terokk.jpg\\n| titles = The Talon King\\n| gender = Male\\n| level = 72\\n| type = Elite\\n| race = Arakkoa\\n| creature = Undead\\n| faction = Combat\\n| aggro = {{aggro|-1|-1}}\\n| foccupation = Talon King of the Arakkoa, King of the Outcasts and Founder of Skettis\\n| status = Deceased (lore)<br/> Killable {{bc-inline}}\\n| location = [[Terokk\'s Rest]] in [[Skettis

In [55]:
WORD_RE = re.compile(r"\[{2}.*?\]{2}")

for word in WORD_RE.findall(wikitext):
    word = word[2:-2].replace(" ", "_")
    print(word)

Terokk's_Rest
Skettis
Terokkar_Forest
Lithic
arakkoa
Rukhmar
Skettis
Talon_King_Ikiss
kaliri
Lower_City
File:Terrok_Battle.png|thumb|left|Terokk_doing_battle_with_the_Bloodmane_saberon,_before_becoming_cursed.
Image:Avatar_of_Terokk.jpg|thumb|The_Avatar_of_Terokk
Dark_Portal
World_of_Warcraft:_Chronicle_Volume_2
spires_of_Arak
Hidden_in_Plain_Sight
Lithic
Warp_Stalker|stalkers
Wind_Serpent|wind_serpents
Colossus|colossi
Adversarial_Blood
Pridelord_Karash
Bloodmane_tribe|Bloodmane
saberon
Rukhmar
The_Talon_King
Pridelord_Karash
Skyreach
Apexis
Sethekk_Hollow
Sethe
Adherents_of_Rukhmar
Anzu
dread_raven
The_Eye_of_Anzu
Void|dark
arcane
Talonpriest
Skettis
Arakkoa_Outcasts
Talador
Terokkar_Forest
Dreamcatcher
Beakbreaker_of_Terokk
Hazzik's_Bargain
Reshad
Terokk's_Fall
Isfar
Countdown_to_Doom
Veil_Shienor
Veil_Reskk
The_Eyes_of_Skettis
Veil_Skith
Darkstone_of_Terokk
Veil_Skith:_Darkstone_of_Terokk
Veil_Rhaze
Veil_Rhaze:_Unliving_Evil
Veil_Lithic
kaliri
Veil_Lithic:_Preemptive_Strike
Veil_Sh